<a href="https://colab.research.google.com/github/snpushpi/Covid19_Contact_tracing_repo_beta/blob/master/Final_No_Fake_path_contact_tracing_different_hash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import collections
import hashlib
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import random
from collections import Counter

In [2]:
import pandas as pd
import os

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Modified_Final_Dataset1.csv to Modified_Final_Dataset1.csv


In [4]:
import io

In [5]:
new_dataframe= pd.read_csv(io.BytesIO(uploaded['Modified_Final_Dataset1.csv']))

In [7]:
new_dataframe[:20]

,Unnamed: 0,Unnamed: 0.1,A,B,C,D,E,F,G,Latitude,Longitude,Timestamp,User_ID,Intersection determiner,Modified_Latitude,Modified_Longitude
0,0,0,0,0,0,0,0,0,0,39.9762,116.3305,0,57,Querier himself,39.9762,116.3305
1,1,1,1,1,1,1,1,1,1,39.9761,116.3304,5,57,Querier himself,39.9761,116.3304
2,2,2,2,2,2,2,2,2,2,39.9758,116.3299,10,57,Querier himself,39.9758,116.3299
3,3,3,3,3,3,3,3,3,3,39.9761,116.3303,15,57,Querier himself,39.9761,116.3303
4,4,4,4,4,4,4,4,4,4,39.9762,116.3305,20,57,Querier himself,39.9762,116.3305
5,5,5,5,5,5,5,5,5,5,39.9763,116.3304,25,57,Querier himself,39.9763,116.3304
6,6,6,6,6,6,6,6,6,6,39.9763,116.3303,30,57,Querier himself,39.9763,116.3303
7,7,7,7,7,7,7,7,7,7,39.9761,116.3301,35,57,Querier himself,39.9761,116.3301
8,8,8,8,8,8,8,8,8,8,39.9753,116.3308,40,57,Querier himself,39.9753,116.3308
9,9,9,9,9,9,9,9,9,9,39.9753,116.3310,45,57,Querier himself,39.9753,116.3310


create a dataframe object from the csv file

Next few parts, we will be making our data in a usable format for feeding into our dataset

In [8]:
def list_generate(new_dataframe,user_num):    
    user_lat = new_dataframe[new_dataframe.User_ID==user_num]['Modified_Latitude'].to_list()
    user_lon = new_dataframe[new_dataframe.User_ID==user_num]['Modified_Longitude'].to_list()
    user_timestamp = new_dataframe[new_dataframe.User_ID==user_num]['Timestamp'].to_list()
    l = len(new_dataframe[new_dataframe.User_ID==user_num])
    user_path = [(user_lat[i], user_lon[i], user_timestamp[i]) for i in range(l)]
    return user_path


For 50 infected users, we will create a list of lists where each list is the path of one infected patient

In [9]:
querier_path = list_generate(new_dataframe,57)
infected_list = []
for i in range(50):
    infected_list.append(list_generate(new_dataframe,i))
    

In [10]:
min_lat = 23.4377
max_lat = 40.9999
min_lon = 113.7947
max_lon = 117.1832

In [11]:
def position_determiner(lat, lon):
    '''Given a latitude, longitude co-ordinate, we have to find the small universe it belongs to.'''
    k = math.floor((lat-min_lat)/0.0030)
    j = math.floor((lon-min_lon)/0.0030)
    return (k,j)
    

In [12]:
def path_divider(path):
    '''given a path (a list of tuples, (lat, lon, timestamp)), divides it into zones.
    returns: a dictionary zone->[{timestamp->(lat,lon, timestamp)},[timestamp_list]]'''
    track_dict = {}
    for elt in path:
        temp_val = position_determiner(elt[0], elt[1])
        if temp_val not in track_dict:
            track_dict[temp_val]=[{},[]]
            track_dict[temp_val][0][elt[2]]= elt
            track_dict[temp_val][1].append(elt[2])
        else:
            track_dict[temp_val][0][elt[2]]= elt
            track_dict[temp_val][1].append(elt[2])
    return track_dict


In [13]:
class Querier():
    def __init__(self,true_path):
        self.true_path = true_path
        self.filtered_dictionary = {}
        self.intersection = []

    def filter_divider(self):
        '''returns zone->timestamp_list dictionary.'''
        self.filtered_dictionary = path_divider(self.true_path)     
        print(self.filtered_dictionary)
        filt_dict = {e:self.filtered_dictionary[e][1] for e in self.filtered_dictionary}
        return filt_dict

    def checker(self, freq_dict, zone):
        '''given the dictionary, checks the intersection with its elements, in other words, for each of its elements in this zone,
        it checks whether the frequency in the dictionary is greater than 1.'''
        timestamp_list = self.filtered_dictionary[zone][1]
        new_all_list = [zone+(timestamp,) for timestamp in timestamp_list]
        for elt in new_all_list:
            if elt in freq_dict:
                if math.floor(freq_dict[elt])>=1:
                    self.intersection.append(elt)
                
    def announcer(self):
        '''returns the self.intersection list.'''
        result_list=[]
        for elt in self.intersection:
            result_list.append((elt[0]*0.0030+min_lat,elt[1]*0.0030+min_lon,elt[2]))
        return result_list


In [33]:
class Infected():
    def __init__(self, true_path,width, depth,epsilon):
        self.width = width
        self.depth = depth
        self.epsilon = epsilon
        self.true_path = true_path
        self.filtered_dictionary = {}
        self.new_level2_filter = {}
    #Implementing the client side algorithm which works on each particular data element. First it selects an integer randomly from 0 to depth-1, 
    #we store that value as j, next, we initialize a vector of -1, of length width, and set that jth index as 1. Then we create another vector of -1s
    #and +1s, with fixed probabilities determined by the parameter epsilon. Then we do element wise multioplication and return the 
    #result vector. 

    def Client_Side(self,d, seed):
        '''parameters are a data element, epsilon value and a hash family, but we are using mmh3 here so didn't add that as a parameter'''
        j = np.random.randint(self.depth) 
        z_vect = np.zeros((1,self.width))
        v = z_vect-1 
        hex_num = hashlib.sha256((str(seed[j])+str(d)).encode('utf-8')).hexdigest()
        int_hex = int(hex_num[:5],16)
        index = int_hex % self.width
        v[0,index]=1
        val = np.exp(self.epsilon/2)
        probability_of_1 = val/(val+1) 
        probability_of_neg_1 = 1/(val+1)
        b = np.random.choice([1,-1],self.width,p=[probability_of_1,probability_of_neg_1]) 
        final_vector = v*b
        return (final_vector,j) 

    def send_flipped_bit_vector(self,zone,seed):
        result_list = []
        for elt in self.new_level2_filter[zone]:
            result_list.append(self.Client_Side(elt,seed))
        return result_list

    def filter_divider(self):
        '''returns: zone->timestamp_list.'''
        self.filtered_dictionary = path_divider(self.true_path)
        filt_dict = {e:self.filtered_dictionary[e][1] for e in self.filtered_dictionary}
        return filt_dict
        
    def filter2(self,filtered_dict_from_server):
        '''filtered_dict_from_server, a dictionary with zone->timestamps.
        modifies: a new_level2_filter list, and new_level2_filter_list[zone] is a list of (lat, lon, timestamp).'''
        for zone in filtered_dict_from_server:
            self.new_level2_filter[zone]=[]
            for elt in filtered_dict_from_server[zone]:
                self.new_level2_filter[zone].append(self.filtered_dictionary[zone][0][elt])



In [41]:
class Server():
    def __init__(self, width, depth, epsilon):
        self.width = width
        self.depth = depth
        self.epsilon = epsilon
        self.zone_list=[]
        self.new_filtered_list = []

    def Compute_Sketch_Matrix(self,D, seed):
        '''So each element of D is a tuple where the first element is v_i and second element is j_i(which is created by the above funstion after getting 
        passed by Client_side each time), we have privacy parameter epsilon and dimensions. v_i is the a vector an it has the sma eshape as the 
        vectors returned by the Client_side algorithm'''
        val = np.exp(self.epsilon/2)
        n = len(D)
        c_epsilon = (val+1)/(val-1) 
        vec_one = np.ones((1,self.width))
        manipulated_data_matrix = np.zeros((n,self.width)) #Creatimg a matrix for x_is
        for elt in enumerate(D):
            new_vect = elt[1][0].reshape((1,self.width))
            man_vect = (c_epsilon/2)*new_vect #scalar and vector multiplication 
            half_vec_one = 0.5*vec_one 
            sum_vect = man_vect + half_vec_one
            manipulated_data_matrix[[elt[0]],:]= self.depth*sum_vect  
        M = np.zeros((self.depth,self.width))
        for elt in enumerate(D):
            for l in range(self.width): 
                M[elt[1][1],l]=M[elt[1][1],l]+manipulated_data_matrix[elt[0],l]
        return M 

    def frequency_counter(self, Sketch_Matrix, total_path_length, universe_list, seed):
        ''' we get a sketch matrix and we try to get the frequencies for all elements in the true_path.'''
        frequency_dict = {}
        for d in universe_list:
            n = total_path_length
            frac1 = self.width/(self.width-1)
            frac2 = n/self.width
            row_sum = 0
            for i in range(self.depth):
                hex_num = hashlib.sha256((str(seed[i])+str(d)).encode('utf-8')).hexdigest()
                int_hex = int(hex_num[:5],16)
                index = int_hex % self.width
                row_sum = row_sum + Sketch_Matrix[i,index]
            avg_row_sum = row_sum/self.depth 
            subtraction = avg_row_sum - frac2 
            assumed_freq = frac1*subtraction 
            frequency_dict[d]=assumed_freq
        return frequency_dict

    def checker(self,querier_filtered_path, infected_patients_filtered_dict_list):
        '''gets zone-> timestamps and returns list of filtered zone-> set(timestamps) dictionaries.'''
        l = len(infected_patients_filtered_dict_list)
        for i in range(l):
            temp_dict={}
            for zone in infected_patients_filtered_dict_list[i]:
                if zone in querier_filtered_path:
                    temp_dict[zone] = set(infected_patients_filtered_dict_list[i][zone]) & set(querier_filtered_path[zone])
            self.new_filtered_list.append(temp_dict)#For all infected patients, it maintains a dictionary, mapping zone to timestamps.
        return self.new_filtered_list
    
    def universe_generate(self, zone, timestamp_list):
        '''generate the universe list with the timestamps.'''
        lat = zone[0]
        lon = zone[1]
        lat_list = []
        lon_list = []
        universe_list = []
        for i in np.arange(lat, lat+0.0031, 0.0001):
            for j in np.arange(lon, lon+0.0031, 0.0001):
                for time in timestamp_list:
                    universe_list.append((i, j, time))
        return universe_list

    def bit_flip_caller(self, infected_patients, querier, run_number):
        '''Will call the flip bit method on all zones of each infected patient.'''
        zone_universe_list = set()
        l = len(self.new_filtered_list)
        for i in range(l):
            zone_universe_list|=set(self.new_filtered_list[i].keys())
        big_timestamp_dictionary = {e:set() for e in zone_universe_list}
        big_tracker_dictionary = {e:[] for e in zone_universe_list}#zone-> infected person who is in that zone
        #generating zone->timestamps for all infected patient dictionary.
        for zone in zone_universe_list:
            for i in range(l):
                if zone in self.new_filtered_list[i]:
                    big_timestamp_dictionary[zone]|=set(self.new_filtered_list[i][zone])
                    big_tracker_dictionary[zone].append(i)
        for zone in zone_universe_list:
            universe_list = self.universe_generate(zone, big_timestamp_dictionary[zone])
            freq_cum_dict = {d:0 for d in universe_list}
            for i in range(run_number):
                big_vect_list = []
                seed = np.random.choice(200, size=150, replace= False)
                #call a function which does some calculation
                for j in big_tracker_dictionary[zone]:
                    big_vect_list+=infected_patients[j].send_flipped_bit_vector(zone,seed)
                n = len(big_vect_list)
                M = self.Compute_Sketch_Matrix(big_vect_list,seed)
                freq_dict = self.frequency_counter(M,n, universe_list,seed)
                for elt in freq_dict:
                    freq_cum_dict[elt]=(i*freq_cum_dict[elt]+freq_dict[elt])/(i+1)
            querier.checker(freq_cum_dict, zone)


In [ ]:
#in the new mode, we invoke this method and the server receives this flipped bit vectors, as we want to average the 
#bit flipping thing, we run this process a mentioned number of times. and make compute sketch matrix.

In [44]:
def driver_code(width, depth, epsilon, run_number):
    '''1. Create instances of querier, server and infected patients.
       2. Call the filter method on all the participants in the server, so all the querier and the infectd patients filter out their paths 
          in multiple universes. This function will output a dictionary kepys mapping to 2 lists, one list is the co-ordinates in that universe and another list is the timestamp.
       3. Now send this filtered timestamp and zone list to the server. Form can be (zone code->timestamps in that zone)
       4. Taking the querier's timestamp against the infected patient's timestamp, server will level2 filtering and send similar form of 
          dictionary to all the infected patients and the querier, so basically finding the timestamp intersection between querier and infected patient pair
          and server is keeping the lists for his calculation.
       5. The participants, upon receiving the information, filters out the information further and start processing data from their end, aka, 
          calls the server caller method.(So in our implementation, we will be processing all infected patients one by one)
       6. The server caller method, is a common method which will be used in other parts too, when this method is called from some 
          infected patient, it asks for flipped vectors a mentioned number of times from the infected patient and upon receiving it every time,
          the server does the compute sketch matrix operation and contnues averaging it and finally gets the aggregated sketch matrix, as the server knows which 
          infected patient and from which zone, it was being called, he finally calls freq generator with this information.(Zone code, sketch matrix and server has saved the timestamp info
          for generating universe list)
       7. Thus for each caller method, server keeps generating frequency histogram and keeps aggregating it and will finally send it to the querier 
       
    '''
    querier = Querier(querier_path)
    infected_patients = []
    for infected in infected_list:
        infected_patients.append(Infected(infected, width, depth, epsilon))
    server_instance = Server(width, depth, epsilon)
    infected_patients_filtered_dict_list=[]
    for e in infected_patients:
        infected_patients_filtered_dict_list.append(e.filter_divider())
    querier_filter = querier.filter_divider()
    new_filtered_list = server_instance.checker(querier_filter, infected_patients_filtered_dict_list)
    l = len(new_filtered_list)
    for i in range(l):
        infected_patients[i].filter2(new_filtered_list[i])
    server_instance.bit_flip_caller(infected_patients, querier, run_number)
    return querier.announcer()



In [45]:
width = 150
depth = 100
epsilon = 0.45
run_number = 10
driver_code(width, depth, epsilon, run_number)

{(5512, 845): [{0: (39.9762, 116.3305, 0), 5: (39.9761, 116.3304, 5), 10: (39.9758, 116.3299, 10), 15: (39.9761, 116.3303, 15), 20: (39.9762, 116.3305, 20), 25: (39.9763, 116.3304, 25), 30: (39.9763, 116.3303, 30), 35: (39.9761, 116.3301, 35), 40: (39.9753, 116.3308, 40), 45: (39.9753, 116.331, 45), 50: (39.9752, 116.3313, 50)}, [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50]], (5512, 846): [{55: (39.9752, 116.3335, 55), 60: (39.9752, 116.3343, 60)}, [55, 60]], (5512, 847): [{65: (39.9753, 116.3375, 65), 70: (39.9753, 116.3379, 70), 75: (39.9753, 116.3382, 75), 80: (39.9753, 116.3384, 80), 85: (39.9754, 116.3385, 85), 280: (39.9766, 116.338, 280), 305: (39.9766, 116.3375, 305), 345: (39.9763, 116.3385, 345), 350: (39.9761, 116.3384, 350), 355: (39.9759, 116.3384, 355), 360: (39.9757, 116.3384, 360), 365: (39.9754, 116.3383, 365), 370: (39.9752, 116.3381, 370), 375: (39.9752, 116.3383, 375), 380: (39.9751, 116.3384, 380), 385: (39.9752, 116.3386, 385), 390: (39.9752, 116.3384, 390), 395: (39

[(39.9767, 116.3387, 160),
 (39.9527, 116.3477, 1910),
 (39.9797, 116.3387, 2710),
 (39.9737, 116.3297, 20),
 (39.9737, 116.3297, 25),
 (39.9737, 116.3297, 40),
 (39.9797, 116.3417, 2785),
 (39.9797, 116.3447, 1860),
 (39.9797, 116.3447, 1865),
 (39.9737, 116.3387, 90),
 (39.9737, 116.3387, 110),
 (39.9737, 116.3387, 115),
 (39.9737, 116.3387, 120),
 (39.9737, 116.3357, 75),
 (39.9737, 116.3357, 80),
 (39.9737, 116.3357, 85),
 (39.9737, 116.3357, 345),
 (39.9737, 116.3357, 365),
 (39.9737, 116.3357, 375),
 (39.9737, 116.3357, 380),
 (39.9737, 116.3357, 395),
 (39.9737, 116.3357, 410),
 (39.9737, 116.3357, 415),
 (39.9737, 116.3357, 420)]